# A Quantitative Analyst's Workflow for AAPL

---

## Section 1 : Why??

Sebelum kita analisis dan buat model, kita perlu ngerti dulu, kenapa kita mau prediksi harga saham AAPL??

### 1.1. Tujuan - Capital Gain & Risk Management

balik ke dasar dalam finance dan investment, invest ke capital market itu tujuannya yang gedein uang kita. nah model disini dipake untuk bantu analisis informasi atau data yang ada dengan lebih efisien dan otomatis. of course model gaakan sempurna, karena pada dasarnya sulit untuk mebemukan pola dalam human behavior penggerak ekonomi. tapi setidaknya ini bisa dijadikan bantuan dalam management investment portofolio kita.

### 1.2. Apa yang Mau di Prediksi??

Kita mau bikin model yang bisa prediksi close value dari AAPL keesokan harinya.  secara simpel itu kaya gini

$Prediksi Harga Besok = fungsi(Data Haei Ini)$

atau :

$Predicted\_Close(t+1) = f(X_t)$



Btw, kita cuma pake data sampe hari ini buat nebak harga besok. Gak boleh tau data besok buat bisa prediksi hari ini, ini disebut lookahead bias, dan ini haram hukumnya di finance modelling.

so, fitur yang kita pake untuk prediksi adalah

- harga pembukaan, tertinggi, terendah, close, dan volume (OHCLC).
- semua data diambil dari data hari hari sebelumnya, sampai hari ini.

### 1.3. Datanya dari mana?? `yfinance` jawabannya

Kita akan menggunakan data harian OHLCV (Open, High, Low, Close, Volume) untuk saham 'AAPL', menggunakan `yfinance`. kita bakal ambil data selama 5 tahun terakhir, kenapa 5 tahun?. well, 5 tahun adalah waktu yang udah cukup untuk dapet pola perubahan pasar, naik turun, big event kaya covid dan kenaikan suku bunga. dan juga ga terlalu lama yang bikin data ga relevan ikut masuk dan bikin model bias atau pusing nangkep pola.

#### Dependencies 


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
import shap
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)

print("Semua library berhasil di-import.")

/Users/ryanakmalpasya/Documents/[1] BS/[2] Freelance/[3] PROJECTS/:dev:projects:ai: /Stock Price Predictor/myvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Semua library berhasil di-import.


#### Data

In [5]:
# ticker dan periode waktu
ticker = 'AAPL'
start_date = '2019-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')

try:
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        raise ValueError("Datanya kosong. Mungkin ticker-nya salah atau gak ada data.")
    print(f"Data {ticker} berhasil diambil dari {start_date} sampai {end_date}.")
    print("Informasi umum data:")
    data.info()
except Exception as e:
    print(f"Error ambil data: {e}")

# Lihat isi data
print("\nLima baris pertama:")
display(data.head())

print("\nStatistik umum:")
display(data.describe())

/var/folders/fz/vv4pwkl920qbrm78wf1yyykr0000gn/T/ipykernel_21427/3863097484.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Data AAPL berhasil diambil dari 2019-01-01 sampai 2025-07-31.
Informasi umum data:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1653 entries, 2019-01-02 to 2025-07-30
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, AAPL)   1653 non-null   float64
 1   (High, AAPL)    1653 non-null   float64
 2   (Low, AAPL)     1653 non-null   float64
 3   (Open, AAPL)    1653 non-null   float64
 4   (Volume, AAPL)  1653 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 77.5 KB

Lima baris pertama:


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2019-01-02,37.617859,37.839395,36.738870,36.896088,148158800
2019-01-03,33.870838,34.711713,33.825578,34.297229,365248800
2019-01-04,35.316761,35.385843,34.254354,34.428245,234428400
2019-01-07,35.238148,35.452537,34.754585,35.421569,219111200
2019-01-08,35.909904,36.164789,35.378701,35.626436,164101200



Statistik umum:


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
count,1653.000000,1653.000000,1653.000000,1653.000000,1.653000e+03
mean,141.943203,143.392510,140.335096,141.795834,9.090863e+07
std,56.667378,57.149589,56.122785,56.621454,5.157802e+07
min,33.870838,34.711713,33.825578,34.297229,2.323470e+07
25%,108.883003,110.309660,106.730760,109.038826,5.527590e+07
50%,146.816757,148.404486,145.317234,146.473738,7.775800e+07
75%,181.424316,183.314295,179.913701,181.255329,1.090120e+08
max,258.396667,259.474086,257.010028,257.568678,4.265100e+08
